In [2]:
from pandas import DataFrame
from pandas import concat
import pandas as pd

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [14]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [19]:
df = pd.read_csv('../data/Neighbourhood_model_data.csv')

In [20]:
df = df[df['NEIGHBOURHOOD_NAME'] == 'Downtown']

In [21]:
df = df[['INTEREST_RATE','CURRENT_MEDIAN_PRICE']]

In [23]:
data = series_to_supervised(df,2)
print(data)

    var1(t-2)  var2(t-2)  var1(t-1)  var2(t-1)   var1(t)  var2(t)
15   6.104167      0.362   4.729167      0.411  2.395833    0.418
16   4.729167      0.411   2.395833      0.418  2.604167    0.417
17   2.395833      0.418   2.604167      0.417  3.000000    0.444
18   2.604167      0.417   3.000000      0.444  3.000000    0.462
19   3.000000      0.444   3.000000      0.462  3.000000    0.461
20   3.000000      0.462   3.000000      0.461  3.000000    0.450
21   3.000000      0.461   3.000000      0.450  2.775000    0.462
22   3.000000      0.450   2.775000      0.462  2.700000    0.503
23   2.775000      0.462   2.700000      0.503  2.700000    0.658
24   2.700000      0.503   2.700000      0.658  2.700000    0.743
25   2.700000      0.658   2.700000      0.743  2.700000    0.793


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 13 to 25
Data columns (total 5 columns):
NEIGHBOURHOOD_NAME       13 non-null object
TAX_ASSESSMENT_YEAR      13 non-null int64
PREVIOUS_MEDIAN_PRICE    13 non-null float64
INTEREST_RATE            13 non-null float64
CURRENT_MEDIAN_PRICE     13 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 624.0+ bytes


In [24]:
# split into train and test sets
values = data.values
n_train_hours = 9
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(9, 1, 5) (9,) (2, 1, 5) (2,)


In [28]:
print(train_X)

[[[6.10416667 0.362      4.72916667 0.411      2.39583333]]

 [[4.72916667 0.411      2.39583333 0.418      2.60416667]]

 [[2.39583333 0.418      2.60416667 0.417      3.        ]]

 [[2.60416667 0.417      3.         0.444      3.        ]]

 [[3.         0.444      3.         0.462      3.        ]]

 [[3.         0.462      3.         0.461      3.        ]]

 [[3.         0.461      3.         0.45       2.775     ]]

 [[3.         0.45       2.775      0.462      2.7       ]]

 [[2.775      0.462      2.7        0.503      2.7       ]]]
